In [1]:
import pandas as pd
from itertools import combinations
from collections import Counter
import networkx as nx
import json
from tqdm import tqdm

c:\Users\aneas\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


[Get dataset here](https://nijianmo.github.io/amazon/index.html)

### Filtering

In [ ]:
df = pd.read_json('Movies_and_TV_5.json', lines = True)
df = df.loc[df['verified']]
df.drop(['reviewTime', 'image', 'vote', 'style', 'verified'], inplace=True, axis=1)


In [12]:
df.head()

,overall,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime
0,5,A2M1CU2IRZG0K9,0005089549,Terri,So sorry I didn't purchase this years ago when...,Amazing!,1352419200
1,5,AFTUJYISOFHY6,0005089549,Melissa D. Abercrombie,Believe me when I tell you that you will recei...,Great Gospel VHS of the Cathedrals!,1325203200
2,5,A3JVF9Y53BEOGC,000503860X,Anthony Thompson,"I have seen X live many times, both in the ear...",A great document of a great band,1114041600
3,5,A12VPEOEZS1KTC,000503860X,JadeRain,"I was so excited for this! Finally, a live co...",YES!! X LIVE!!,1112745600
4,5,ATLZNVLYKP9AZ,000503860X,T. Fisher,X is one of the best punk bands ever. I don't ...,X have still got it,1291334400


In [11]:
df.to_csv("tv.csv")

### Network

In [2]:
df = pd.read_csv('tv.csv')
df.drop(['Unnamed: 0'], inplace=True, axis=1)

In [3]:
grouped = df.groupby('asin')['reviewerID'].unique()
len(grouped)

59518

In [13]:
len(grouped.iloc[350])

540

In [15]:
edges = Counter()
for reviewers in tqdm(grouped.values, desc="Movies"):
    for pair in combinations(reviewers, 2):
        edges[pair] += 1

Movies:  90%|█████████ | 53620/59518 [1:25:52<09:26, 10.41it/s]   


MemoryError: 

In [ ]:
edgelist = [(a, b, count) for (a, b), count in edges.items()]
len(edgelist)

In [ ]:
df_edges = pd.DataFrame(
    [(u, v, w) for (u, v), w in edges.items()],
    columns=['reviewer1', 'reviewer2', 'weight']
)
df_edges.to_csv('edges.csv', index=False)

### Saving graph

In [ ]:
def save_graph(graph_file, G):
    data = nx.readwrite.json_graph.node_link_data(G)
    with open(graph_file, "w") as f:
        json.dump(data, f)

def load_graph(graph_file):
    with open(graph_file, "r") as f:
        data = json.load(f)
    return nx.readwrite.json_graph.node_link_graph(data)